In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 999)

In [3]:
tweets = pd.read_csv('twcs/twcs.csv')
tweets.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist you. We would need to get you into a private secured link to further assist.,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messages and no one is responding as usual,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so that we can further assist you. Just click ‘Message’ at the top of your profile.,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [4]:
tweets.dtypes

tweet_id                     int64
author_id                   object
inbound                       bool
created_at                  object
text                        object
response_tweet_id           object
in_response_to_tweet_id    float64
dtype: object

In [5]:
tweets_2 = tweets[['text','tweet_id']]
tweets.loc[pd.isna(tweets['response_tweet_id']),'response_tweet_id']=0
tweets['comma_count'] = tweets.response_tweet_id.str.count(',')
tweets=tweets.loc[tweets.comma_count==0]
tweets['response_tweet_id'] = tweets['response_tweet_id'].astype('int')

In [6]:
tweets.shape

(1548719, 8)

In [7]:

merged_tweets = pd.merge(tweets[['response_tweet_id','text']], tweets_2,left_on='response_tweet_id', right_on='tweet_id')

In [8]:
merged_tweets.shape

(1548719, 4)

In [9]:
merged_tweets.columns=['response_tweet_id','text','response','tweet_id']
merged_tweets.head()

,response_tweet_id,text,response,tweet_id
0,2,@115712 I understand. I would like to assist you. We would need to get you into a private secured link to further assist.,@sprintcare and how do you propose we do that,2
1,1,@sprintcare I have sent several private messages and no one is responding as usual,@115712 I understand. I would like to assist you. We would need to get you into a private secured link to further assist.,1
2,3,@115712 Please send us a Private Message so that we can further assist you. Just click ‘Message’ at the top of your profile.,@sprintcare I have sent several private messages and no one is responding as usual,3
3,4,@sprintcare I did.,@115712 Please send us a Private Message so that we can further assist you. Just click ‘Message’ at the top of your profile.,4
4,12,"@115713 We understand your concerns and we'd like for you to please send us a Direct Message, so that we can further assist you. -AA",@sprintcare You gonna magically change your connectivity for me and my whole family ? 🤥 💯,12


In [10]:
import re
def remove_spcl(x):
    return  re.sub(r"\s+\s+"," ",re.sub(r"[^A-Za-z0-9()!?\'\`\"\r+\n+]"," ",x.lower())).strip()

In [11]:
merged_tweets['text']=merged_tweets['text'].apply(remove_spcl)
merged_tweets['response']=merged_tweets['response'].apply(remove_spcl)
merged_tweets.head()

,response_tweet_id,text,response,tweet_id
0,2,115712 i understand i would like to assist you we would need to get you into a private secured link to further assist,sprintcare and how do you propose we do that,2
1,1,sprintcare i have sent several private messages and no one is responding as usual,115712 i understand i would like to assist you we would need to get you into a private secured link to further assist,1
2,3,115712 please send us a private message so that we can further assist you just click message at the top of your profile,sprintcare i have sent several private messages and no one is responding as usual,3
3,4,sprintcare i did,115712 please send us a private message so that we can further assist you just click message at the top of your profile,4
4,12,115713 we understand your concerns and we'd like for you to please send us a direct message so that we can further assist you aa,sprintcare you gonna magically change your connectivity for me and my whole family ?,12


In [76]:
merged_tweets[:929231][['text','response']].to_csv('cleaned_tweets.csv')

In [71]:
merged_tweets.shape

(1548719, 4)

In [72]:
0.6*1548719

929231.4

In [81]:
0.8*1548719

1238975.2

In [80]:
merged_tweets.shape

(1548719, 4)

In [20]:
merged_tweets.loc[:929230,['text','response']].loc[~pd.isna(merged_tweets.response)].to_csv('twitter_data/cleaned_tweets_train.csv',index=False)

In [21]:
merged_tweets.loc[929231:1238975,['text','response']].loc[~pd.isna(merged_tweets.response)].to_csv('twitter_data/cleaned_tweets_valid.csv',index=False)

In [22]:
merged_tweets.loc[1238975:,['text','response']].loc[~pd.isna(merged_tweets.response)].to_csv('twitter_data/cleaned_tweets_test.csv',index=False)